   # Unit Tests for Non-Dominated Sorting Genetic Algorithm II (NSGAII)

In [ ]:
import unittest
import random
from pto.solvers.NSGAII import (
    NSGAII,
    BinaryProblem,
) 


class TestNSGAII(unittest.TestCase):
    def setUp(self):
        """Set up a small binary problem for testing."""
        self.problem = BinaryProblem(n_dimensions=3)
        self.nsga2 = NSGAII(
            self.problem,
            n_generations=2,
            population_size=4,
            mutation_rate=0.1,
            crossover_rate=0.9,
            better=min,
        )

    def test_initialization(self):
        """Test the NSGAII initialization."""
        self.assertEqual(self.nsga2.n_generations, 2)
        self.assertEqual(self.nsga2.population_size, 4)
        self.assertEqual(self.nsga2.mutation_rate, 0.1)
        self.assertEqual(self.nsga2.crossover_rate, 0.9)

    def test_create_population(self):
        """Test if the initial population is generated correctly."""
        population = self.nsga2.create_population()
        print(f"Population {population}")
        self.assertEqual(len(population), 4)
        for ind in population:
            self.assertEqual(len(ind), 3)
            self.assertTrue(all(bit in [0, 1] for bit in ind))

    def test_evaluate_population(self):
        """Test if the evaluation function returns the correct fitness values."""
        population = [[1, 0], [1, 1], [0, 0]]
        fitnesses = self.nsga2.evaluate_population(population)
        self.assertEqual(fitnesses, [(1, 1), (2, 0), (0, 2)])

    def test_dominates(self):
        """Test the dominance relationship."""
        self.assertTrue(self.nsga2.dominates((1, 2), (2, 2)))
        self.assertFalse(self.nsga2.dominates((2, 2), (1, 2)))
        self.assertFalse(self.nsga2.dominates((2, 2), (2, 2)))

    def test_fast_non_dominated_sort(self):
        """Test the non-dominated sorting algorithm."""
        population = [[1], [2], [3], [4], [5]]
        fitnesses = [(1, 1), (2, 0), (0, 2), [1, 0], [0, 0]]
        fronts = self.nsga2.fast_non_dominated_sort(population, fitnesses)

        self.assertEqual(len(fronts), 3)
        self.assertEqual(set(fronts[0]), {4})
        self.assertEqual(set(fronts[1]), {3, 2})
        self.assertEqual(set(fronts[2]), {1, 0})

        
    def test_create_offspring(self):
        """Test offspring generation with crossover and mutation."""
        
        population = [[0, 1, 0], [1, 0, 1]]
        offspring = self.nsga2.create_offspring(population)
        self.assertEqual(len(offspring), 4)
        for ind in offspring:
            self.assertEqual(len(ind), 3)
            self.assertTrue(all(bit in [0, 1] for bit in ind))

    def test_calculate_crowding_distances(self):
        front = [[1, 2], [3, 4], [2, 3]]
        distances = self.nsga2.calculate_crowding_distances(front)
        
        self.assertEqual(distances[0], float("inf"))
        self.assertEqual(distances[1], float("inf"))
        self.assertEqual(distances[2], 4)

    def test_selection_all_front(self):
        population = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        fitnesses = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)]
        fronts = [[0], [1], [2], [3], [4], [5], [6], [7]] 

        new_pop, new_fit = self.nsga2.selection(population, fitnesses, fronts)
        self.assertEqual(len(new_pop), 4)
        self.assertEqual(new_pop, ['a', 'b', 'c', 'd'])
        self.assertEqual(new_fit, [(1, 1), (2, 2), (3, 3), (4, 4)])

    def test_selection_with_crowding(self):
        self.nsga2.population_size = 5 
        population = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
        fitnesses = [(0, 1), (1, 0), (3, 1), (2.9, 1.1), (1.5, 1.5), (1.1, 2.9), (1, 3), (5, 5), (6, 6)]

        fronts = [[0, 1], [2, 3, 4, 5, 6], [7]]
        new_pop, new_fit = self.nsga2.selection(population, fitnesses, fronts)
        
        self.assertEqual(len(new_pop), 5)
        self.assertEqual(new_pop, ['a', 'b', 'c', 'g', 'e'])
        self.assertEqual(new_fit, [(0, 1), (1, 0), (3, 1), (1, 3), (1.5, 1.5)])
        

    def test_calculate_pareto_front(self):
        population = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
        fitnesses = [(0, 1), (1, 0), (3, 1), (2.9, 1.1), (1.5, 1.5), (1.1, 2.9), (1, 3), (5, 5), (6, 6)]

        p_front, p_front_fitnesses = self.nsga2.calculate_pareto_front(population, fitnesses)

        expected_population = ['a', 'b']
        expected_fitnesses = [(0, 1), (1, 0)]

        sorted_p_front = sorted(tuple(ind) for ind in p_front)
        sorted_expected_pop = sorted(tuple(ind) for ind in expected_population)
        self.assertEqual(sorted_p_front, sorted_expected_pop)
        self.assertEqual(sorted(p_front_fitnesses), sorted(expected_fitnesses))

# Run tests
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

   # Tests for Non-Dominated Sorting Genetic Algorithm II (NSGA-II)

In [ ]:
from pto.solvers import NSGAII as NSGAII

from pto.core.base import Op, tracer, Dist

import random

In [ ]:
def random_program():
    return([tracer.sample('pos 1', Dist(random.random)),
            tracer.sample('pos 2', Dist(random.choice, ['a','b','c'])),
            tracer.sample('pos 3', Dist(random.randint, 1, 10))])

def fitness(sol): return sol[0], sol[2]

In [ ]:
op = Op(generator=random_program, fitness=fitness)

In [ ]:
# instantiate NSGAII
nsgaII=NSGAII(op)

In [ ]:
# NSGAII current parameters
nsgaII.__dict__

In [ ]:
# Execute NSGAII
res=nsgaII()
res

In [ ]:
# result
for i, sol in enumerate(res[1]):
        print(f"\nSolution {i + 1}:")
        print("Pheno:", sol.pheno)
        fit = fitness(sol.pheno)
        print("Fitness:", fit)
        print("Geno:")
        for key, value in sol.geno.items():
            print(f"  {key}: {value}")

In [ ]:
# Test callback with printing
from pprint import pprint

nsgaII = NSGAII(op, callback=print)

pprint(nsgaII.__dict__)

res = nsgaII()

In [ ]:
# Test callback with stopping condition
count = 0
def maxit(_):
    global count
    count += 1
    print(count)
    if count >= 10:
        print('stop!')
        return True

nsga = NSGAII(op, callback=maxit)

pprint(nsga.__dict__)

res = nsga()

for i, sol in enumerate(res[1]):
        print(f"\nSolution {i + 1}:")
        print("Pheno:", sol.pheno)
        fit = fitness(sol.pheno)
        print("Fitness:", fit)
        print("Geno:")
        for key, value in sol.geno.items():
            print(f"  {key}: {value}")
